In [ ]:
%matplotlib widget
import os
import sys
import numpy as np
from scipy.spatial.transform import Rotation
import matplotlib.pyplot as plt
import math

# Helper function for 'axis equal' in matplotlib's 3d view 
def set_3d_axes_equal(ax):
    x_limits = ax.get_xlim3d()
    y_limits = ax.get_ylim3d()
    z_limits = ax.get_zlim3d()
    x_range = abs(x_limits[1] - x_limits[0])
    x_middle = np.mean(x_limits)
    y_range = abs(y_limits[1] - y_limits[0])
    y_middle = np.mean(y_limits)
    z_range = abs(z_limits[1] - z_limits[0])
    z_middle = np.mean(z_limits)
    plot_radius = 0.5*max([x_range, y_range, z_range])
    ax.set_xlim3d([x_middle - plot_radius, x_middle + plot_radius])
    ax.set_ylim3d([y_middle - plot_radius, y_middle + plot_radius])
    ax.set_zlim3d([z_middle - plot_radius, z_middle + plot_radius])    

# Problem definition

Applications:
- Iterative closest point algorithm for point cloud registration
- Robot navigation
- Optimal transformation between two trajectories
- [Comparing moluecular structures](https://cnx.org/contents/HV-RsdwL@23/Molecular-Distance-Measures)

We wish to solve the following non-linear least squares problem:
$$ 
\min_{R,\mathbf{t}} \sum_{i=1}^{N} \lVert(R\mathbf{p}_i + \mathbf{t}) - \mathbf{q}_i\lVert_2,
$$
where $R \in SO(3)$ is the rotation matrix, $\mathbf{t}\in \mathbb{R}^3$ is the translation vector and $\mathbf{p}_i, \mathbf{q}_i \in \mathbb{R}^3$ are the model and target points, respectively. With $\mathbf{r}(R, \mathbf{t}) = (R\mathbf{p}_i + \mathbf{t}) - \mathbf{q}_i$ residual vector function, the problem can be written as:
$$ 
\min_{R,\mathbf{t}} \sum_{i=1}^{N} \mathbf{r}(R,\mathbf{t})^2,
$$
or in vector form:
$$ 
\min_{R,\mathbf{t}} \mathbf{r}^\mathsf{T}(R,\mathbf{t}) \mathbf{r}(R,\mathbf{t}).
$$


To work with some data, we generate random points:

In [ ]:
np.random.seed(10)
pts_model = np.random.rand(10, 3) + np.array([1, 2, 3]) # so the point are not around the origin

Then we need the ground truth rotation and translation values. I parametrize the rotation vector via Euler angles: $R(\gamma, \beta, \alpha)$, where $\gamma$ is the yaw, $\beta$ is the pitch and $\alpha$ is the roll angle. 

In [ ]:
ypr_gt = [45/180.0*math.pi, -15/180.0*math.pi, 20/180.0*math.pi]
t_gt = [1, -2, 3]
x_gt = np.append(ypr_gt, t_gt)

The Euler angles are intrepreted as intrinsic rotations around the Z, Y and final the X axis.

In [ ]:
R_gt = Rotation.from_euler('ZYX', ypr_gt, degrees=False).as_matrix()

 Note that this is the same rotation as defining the extrinsic rotation in X, Y, Z axis order, but roll, pitch, yaw angle order.

In [ ]:
R_int = Rotation.from_euler('ZYX', ypr_gt, degrees=False).as_matrix()
R_ext = Rotation.from_euler('xyz', [ypr_gt[2], ypr_gt[1], ypr_gt[0]], degrees=False).as_matrix()
np.linalg.norm(R_ext - R_int)

Finally we apply the rotation on our synthetic data:

In [ ]:
pts_target = (R_gt @ pts_model.T).T + t_gt

The random model points and the transformet target points look like this in 2D:

In [ ]:
plt.figure(figsize=(6,5))
plt.scatter(pts_model[:, 0], pts_model[:, 1])
plt.scatter(pts_target[:, 0], pts_target[:, 1])

... and in 3D:

In [ ]:
def viz_solution(pts_model, pts_target, pts_model_hat=None):
    plt.figure(figsize=(6,5))
    ax = plt.axes(projection='3d')
    ax.scatter3D(pts_model[:,0], pts_model[:,1], pts_model[:,2], c='r', s=10)
    ax.scatter3D(pts_target[:,0], pts_target[:,1], pts_target[:,2], c='b', s=10)
    if pts_model_hat is not None:
        ax.scatter3D(pts_model_hat[:,0], pts_model_hat[:,1], pts_model_hat[:,2], c='g', s=10)

viz_solution(pts_model, pts_target)

# Solution #1: Singular value decomposition ([Kabsch-Umeyama algorithm](https://en.wikipedia.org/wiki/Kabsch_algorithm))

In his 1978 paper, titled ["A discussion of the solution for the best rotation to relate two sets of vectors"](https://onlinelibrary.wiley.com/doi/abs/10.1107/S0567739478001680), Wolfgang Kabsch gave a closed form solution for the least squares estimate of the rotation, i.e. when $\mathbf{t} = \mathbf{0}$.  However obtaining the translation is somewhat trivial, Umeyama completes Kabsch algorithm in his 1991 paper titled ["Least-squares estimation of transformation parameters between two point patterns"](https://web.stanford.edu/class/cs273/refs/umeyama.pdf). For a more recent derivation, as it appears in modern computer vision courses, see the [lecture notes from ETH Zurich](https://igl.ethz.ch/projects/ARAP/svd_rot.pdf). Note that this is the standard way to solve the problem of finding the optimial transform between two sets of points in least squares sense.

SVD decomposition is the standard and most efficient way to find the rigid body transformation between two point sets.

First we need to compute the center of gravity (or the means) of the two point sets:
$$
\bar{\mathbf{p}} = \frac{\sum_{i=1}^N \mathbf{p}_i}{N}, 
$$
$$
\bar{\mathbf{q}} = \frac{\sum_{i=1}^N \mathbf{q}_i}{N},
$$
and then, move the point sets to their respective center of gravities:
$$
\tilde{\mathbf{p}} = \mathbf{p} - \bar{\mathbf{p}},  
$$
$$
\tilde{\mathbf{q}} = \mathbf{q} - \bar{\mathbf{q}} .
$$

In [ ]:
cog_model = np.mean(pts_model, axis=0)
cog_target = np.mean(pts_target, axis=0)

pts_model_cog = pts_model - cog_model
pts_target_cog = pts_target - cog_target

In [ ]:
C = pts_model_cog.T @ pts_target_cog

In [ ]:
# %% SVD decomposition
[U, S, V] = np.linalg.svd(C)
V = V.T
C_chk = U @ np.diag(S) @ V.T
print('Check decomposition ( =0): ', np.linalg.norm(C_chk - C))

In [ ]:
R_svd = V @ U.T
t_svd = cog_target - R_svd @ cog_model

print('Check R ( =0): ', np.linalg.norm(R_svd - R_gt))
print('Check t ( =0):', np.linalg.norm(t_svd - t_gt))

In [ ]:
pts_model_hat = (R_svd @ pts_model.T).T + t_svd

In [ ]:
res_svd = np.linalg.norm(pts_model_hat - pts_target, axis=1)
print('Check transformation ( =0)', np.linalg.norm(res_svd))

In [ ]:
viz_solution(pts_model, pts_target, pts_model_hat)

Computation time of SVD can be found here: https://www.cs.utexas.edu/users/inderjit/public_papers/HLA_SVD.pdf

# Solution with solving matrix inverse

In [ ]:
from scipy.linalg import sqrtm

We can also obtain the solution using following [formula](https://en.wikipedia.org/wiki/Kabsch_algorithm):
$$
R = (C^\mathsf{T}C)^{1/2}C^{-1}
$$

In [ ]:
R_minv = sqrtm(C.transpose() @ C) @ np.linalg.inv(C)
print(np.linalg.norm(R_gt - R_minv))

# Solution with derivative-free *scipy* function

In [ ]:
from scipy.optimize import minimize

In [ ]:
R_x =  lambda x0: Rotation.from_euler('ZYX', x0[:3], degrees=False).as_matrix()
r_x = lambda x0: ((R_x(x0) @ pts_model.T).T + np.array([x0[3:6]]) - pts_target).ravel()
print('Check residual function ( =0): ', np.linalg.norm(r_x(x_gt)))

We will use the derivative free simplex method.

In [ ]:
x0 = np.array([45.0-15.0, -15.0+25.0, 20.0+25.0, 1.0+1.5, -2.0+1.8, 3.0-1.2]) # converges to the solution in 914 iterations
x0[:3] *= 1.0/180.0*math.pi
#x0 = np.array([0, 0, 0, 0, 0, 0]) # converges to local minumum
    
res = minimize(lambda x: np.linalg.norm(r_x(x)), x0, method='nelder-mead',
               options={'xatol': 1e-6, 'disp': True, 'maxiter': 2000})

print('Check solution ( =0): ', np.linalg.norm(res.x - x_gt))

As we can see this method is relatively sensitive to the choosen initial guess.

However, we can find another derivative-free algorithm that provides the solution even with extreme initial guess. See the example with [BFGS](https://docs.scipy.org/doc/scipy/tutorial/optimize.html#broyden-fletcher-goldfarb-shanno-algorithm-method-bfgs).

In [ ]:
x0 = np.array([0, 0, 0, 0, 0, 0]) # converges to the solution in 49 iterations
res = minimize(lambda x: np.linalg.norm(r_x(x)), x0, method='BFGS',
               options={'disp': True, 'maxiter': 2000})

print('Check solution ( =0): ', np.linalg.norm(res.x - x_gt))

# Solution with Newton's method

In [ ]:
import sympy as sym
from math import cos, sin, pi

In order to improve on the convergence rate and hopefully improve on the sensitivity of the initial guess, let's try to use analytical derivatives.

In [ ]:
o, p, k, x, y, z = sym.symbols('o p k x y z')

In [ ]:
def Rx_s(theta):
  return sym.Matrix([[ 1, 0           , 0           ],
                   [ 0, sym.cos(theta),-sym.sin(theta)],
                   [ 0, sym.sin(theta), sym.cos(theta)]])
  
def Ry_s(theta):
  return sym.Matrix([[sym.cos(theta), 0, sym.sin(theta)],
                   [ 0           , 1, 0           ],
                   [-sym.sin(theta), 0, sym.cos(theta)]])
  
def Rz_s(theta):
  return sym.Matrix([[sym.cos(theta), -sym.sin(theta), 0 ],
                   [ sym.sin(theta), sym.cos(theta) , 0 ],
                   [ 0           , 0            , 1 ]])

ypr2rot_sym = lambda k, p, o: Rz_s(k) @ Ry_s(p) @ Rx_s(o)
#print('Check rotation composition ( =0)', np.linalg.norm(R_fn(ypr_gt[0], ypr_gt[1], ypr_gt[2]) - R_gt))

In [ ]:
R_sym = ypr2rot_sym(k, p, o)
R_sym_sub = R_sym.subs({k: float(ypr_gt[0]), p: float(ypr_gt[1]), o: float(ypr_gt[2])})
R_sym_sub_np = np.array(R_sym_sub.tolist()).astype('float')
print('Check rotation composition ( =0): ', np.linalg.norm(R_sym_sub_np - R_gt))

Define cost function:

In [ ]:
dt_sym = np.tile(np.array([x, y, z]), (pts_model.shape[0], 1))
res_sym_vec = (R_sym @ pts_model.T).T + dt_sym - pts_target
res_sym_vec = res_sym_vec.reshape(pts_model.shape[0]*3, 1) # reshape it from (N x 3) -> (M x 1)
cost_fn_sym = res_sym_vec.T @ res_sym_vec

print('Number of chars of cost function :', len(str(cost_fn_sym)), '  ', len(str(cost_fn_sym))/1000, 'kB')

In [ ]:
cost_fn_sym = sym.simplify(cost_fn_sym) # this might take a bit of time
print('Number of chars of cost function :', len(str(cost_fn_sym)), '  ', len(str(cost_fn_sym))/1000, 'kB')

Define & lambdify gradient function: 

In [ ]:
g = sym.Matrix([cost_fn_sym.diff(var) for var in [k, p, o, x, y, z]])
# g = g.applyfunc(sym.simplify) # does not change anything

g_x_sym = lambda x0: g.subs({k: float(x0[0]), p: float(x0[1]), o: float(x0[2]), 
                             x: float(x0[3]), y: float(x0[4]), z: float(x0[5])})
g_x = lambda x0: np.array(g_x_sym(x0).tolist()).astype('float')
print('Check gradient (= 0): ', np.linalg.norm(g_x(x_gt)))
print('Number of chars of gradient function :', len(str(g)), '  ', len(str(g))/1000, 'kB')

Define & lambdify Hessian matrix: 

In [ ]:
H = sym.hessian(cost_fn_sym, [k, p, o, x, y, z])
#H = H.applyfunc(sym.simplify) # only small imporvement: 7280 vs. 7338 chars

H_x_sym = lambda x0: H.subs({k: float(x0[0]), p: float(x0[1]), o: float(x0[2]), 
                             x: float(x0[3]), y: float(x0[4]), z: float(x0[5])})
H_x = lambda x0: np.array(H_x_sym(x0).tolist()).astype('float')

print('Shape of Hessian (= (6,6)) : ', H_x(x_gt).shape)
print('Min value of Eigenvalues (> 0): ', np.min(np.linalg.eigvals(H_x(x_gt))))
print('Number of chars of Hessian function :', len(str(H)), '  ', len(str(H))/1000, 'kB')

Solve the problem with Newton's method. The method should exhibit quadratic convergence if x0 is "good enough".

In [ ]:
def optimizer_framework(step_function, x0, x_gt):
    print('Iteration # {:6s} {:18s} {:15s} {:20s}'.format('', 'delta gt', 'dx', 'dr'))
    print('----------------------------------------------------------------')
    def print_iter(iter_num, chk, dx, dr):
        print('Iteration #{:d} {:15.8f} {:15.8f} {:15.8f}'.format(iter_num, chk, dx, dr))

    dx = np.inf
    iter_num = 0

    print_iter(iter_num, np.linalg.norm(x0 - x_gt), np.nan, np.linalg.norm(g_x(x0)))
    while np.linalg.norm(dx) > 1e-6 and iter_num < 100:
        iter_num += 1
        dx = step_function(x0)
        x0 = x0 - dx
        print_iter(iter_num, np.linalg.norm(x0 - x_gt), np.linalg.norm(dx), np.linalg.norm(r_x(x0)))
        
    print('Check solution ( =0): ', np.linalg.norm(x0 - x_gt))

In [ ]:
x0 = np.array([45.0-15.0, -15.0+25.0, 20.0+25.0, 1.0+1.5, -2.0+1.8, 3.0-1.2]) # converging to the solution in 8 iterations
#x0 = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0]) # converging to a local minimum in 20 iteration
x0[:3] *= 1.0/180.0*pi

def newton_step(x):
    dx = np.linalg.inv(H_x(x)) @ g_x(x)
    return dx.reshape(-1)

optimizer_framework(newton_step, x0, x_gt)

It seems this solution is also sensitive for the initial guess.

# Solution of the cost with built-in methods

In [ ]:
from scipy.optimize import minimize

For better convergence properties, we can do line search with the Hessian. In the next we will use [built-in](https://docs.scipy.org/doc/scipy/tutorial/optimize.html#newton-conjugate-gradient-algorithm-method-newton-cg) function that utilizes this technique. 

In [ ]:
cost_per_iter = []
def iter_callback(xk):
    dx_gt = np.linalg.norm(xk - x_gt)
    cost_per_iter.append(dx_gt)
    print('Norm from GT: ', dx_gt)

x0 = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0]) # converging to the solution in 37 iterations
res = minimize(lambda x: np.linalg.norm(r_x(x)), x0, method='Newton-CG',
               jac=lambda x: g_x(x).reshape(-1),  hess=H_x,
               options={'xtol': 1e-6, 'disp': True, 'maxiter': 50}, callback=iter_callback)

In [ ]:
plt.figure()
plt.plot(range(len(cost_per_iter)), cost_per_iter, '.-')
plt.yscale('log')
plt.grid(True,which="both", linestyle='-.')
plt.ylabel('log(norm(x_hat - x_gt))')
plt.xlabel('iteration #')
plt.title("Convergence of Newton-CG method")

# Solution with Gauss-Newton method

We should take advantage of the fact that the problem is least squares. This will allow us to use more stable algorithms with faster convergence rate and with less sensitivity to the initial guess.

Create the Jacobian matrix for the least squares problem:

In [ ]:
J = res_sym_vec.jacobian([k, p, o, x, y, z])
J_x_sym = lambda x0: J.subs({k: float(x0[0]), p: float(x0[1]), o: float(x0[2]), 
                             x: float(x0[3]), y: float(x0[4]), z: float(x0[5])})
J_x = lambda x0: np.array(J_x_sym(x0).tolist()).astype('float')
N = J_x(x0).T @ J_x(x0)


print('Shape of Jacobian: ', J.shape)
print('Shape of N: ', N.shape)
print('Number of chars of Jacobian function :', len(str(J)), '  ', len(str(J))/1000, 'kB')

... and the iterations looks more stable:

In [ ]:
x0 = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0]) # converging to the solution in 5 iterations

def gauss_newton_full_step(x):
    dr = r_x(x)
    dx =  np.linalg.inv(J_x(x).T @ J_x(x)) @ J_x(x).T @ dr
    return dx

optimizer_framework(gauss_newton_full_step, x0, x_gt)

Surpresingly, almost quadratic convergence from the beginning with an initial guess far from the solution. However the above approach results in long Jacobian code. Instead we can compute the symbolic Jacobian for each point separatetly.

In [ ]:
m_x, m_y, m_z = sym.symbols('m_x m_y m_z')
t_x, t_y, t_z = sym.symbols('t_x t_y t_z')

m_xyz = np.array([m_x, m_y, m_z])
t_xyz = np.array([t_x, t_y, t_z])

dt_sym_row = np.array([x, y, z])
eq_sym = (R_sym @ m_xyz).T + dt_sym_row - t_xyz

print('Shape of a single point pair equation: (= (3,))', eq_sym.shape)

In [ ]:
J_point_tx = sym.Matrix([eq_sym[0].diff(x) for x in [k, p, o, x, y, z]]).T
J_point_ty = sym.Matrix([eq_sym[1].diff(x) for x in [k, p, o, x, y, z]]).T
J_point_tz = sym.Matrix([eq_sym[2].diff(x) for x in [k, p, o, x, y, z]]).T
J_point = sym.Matrix([J_point_tx, J_point_ty, J_point_tz])

print('Number of chars of Jacobian function :', len(str(J_point)), '  ', len(str(J_point))/1000, 'kB')

In [ ]:
J_point_sym = lambda x0, m_pt, t_pt: J_point.subs({k: float(x0[0]), p: float(x0[1]), o: float(x0[2]), 
                                                   x: float(x0[3]), y: float(x0[4]), z: float(x0[5]),
                                                m_x: float(m_pt[0]), m_y: float(m_pt[1]), m_z: float(m_pt[2]),
                                                t_x: float(t_pt[0]), t_y: float(t_pt[1]), t_z: float(t_pt[2])})

J_point_x = lambda x0, m_pt, t_pt: np.array(J_point_sym(x0, m_pt, t_pt).tolist()).astype('float')

In [ ]:
def build_J(x):
    J_x = np.zeros((pts_model.shape[0]*3, 6))
    for pt_k in range(pts_model.shape[0]):
        J_x[(pt_k*3):(pt_k*3+3)] = J_point_x(x, pts_model[pt_k, :], pts_target[pt_k, :])
    return J_x

print('Check Jacobian composition ( =0): ', np.linalg.norm(build_J(x_gt) - J_x(x_gt)))

In [ ]:
x0 = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0]) # converging to the solution in 5 iterations

def gauss_newton_step(x):
    dr = r_x(x)
    dx = np.linalg.inv(build_J(x).T @ build_J(x)) @ build_J(x).T @ dr
    return dx
    
optimizer_framework(gauss_newton_step, x0, x_gt)

# Solution with least squares and numerical differentiation

In [ ]:
from scipy.optimize import least_squares

There are deravitive-free least squares algorithms that we can utilize to solve these types of problems faster & easier. 

In [ ]:
x0 = np.array([0, 0, 0, 0, 0, 0])
res = least_squares(r_x, x0, verbose=2)

In [ ]:
print('Check solution ( =0): ', np.linalg.norm(res.x - x_gt))

# Solution with automatic differentiation

Follow this link: https://sidsite.com/posts/autodiff/

- Operations are primarly implemented as overloads
- All values are float inside the class
- We provide constant (number) overloads for convenience

In [ ]:
from collections import defaultdict

class Variable(object):
    
    def __init__(self, value, local_gradients=[]):
        self.value = float(value)
        self.local_gradients = local_gradients

    def __float__(self):
         return self.value
    
    def __add__(self, other):
        if not isinstance(other, Variable):
            return self.__add__(Variable(float(other)))        
        value = self.value + other.value    
        local_gradients = ((self, 1.0), (other, 1.0))
        return Variable(value, local_gradients)
    
    def __mul__(self, other):
        if not isinstance(other, Variable):
            return self.__mul__(Variable(float(other)))
        value = self.value * other.value
        local_gradients = ((self, other.value), (other, self.value))
        return Variable(value, local_gradients)

    def __neg__(self):
        value = -1.0 * self.value
        local_gradients = ((self, -1.0),)
        return Variable(value, local_gradients)
    
    def __sub__(self, other):
        if not isinstance(other, Variable):
            return self.__sub__(Variable(float(other)))        
        return self.__add__(other.__neg__())
    
    def __repr__(self):
        return str(self.value) + 'v' # + ' [' + str(self.local_gradients) + ']'

    __radd__  = __add__
    __rmul__ = __mul__
    __rsub__ = __sub__

class VarFunctions(object):
    
    @staticmethod
    def sin(a):
        value = np.sin(a.value)
        local_gradients = (
            (a, np.cos(a.value)),
        )
        return Variable(value, local_gradients)
    
    @staticmethod
    def cos(a):
        value = np.cos(a.value)
        local_gradients = (
            (a, -np.sin(a.value)),
        )
        return Variable(value, local_gradients)

vf = VarFunctions

In [ ]:
def get_gradients(variable):
    
    gradients = defaultdict(lambda: 0)
    
    def compute_gradients(variable, path_value):
        for child_variable, local_gradient in variable.local_gradients:
            # "Multiply the edges of a path":
            value_of_path_to_child = path_value * local_gradient
            # "Add together the different paths":
            gradients[child_variable] += value_of_path_to_child
            # recurse through graph:
            compute_gradients(child_variable, value_of_path_to_child)
    
    compute_gradients(variable, path_value=1)
    # (path_value=1 is from `variable` differentiated w.r.t. itself)
    return gradients

Test a basic operation:

In [ ]:
test_a = Variable(4)
test_b = Variable(4)
test_c = test_a - test_b
print(test_c)

Let's redefine rotations using the newly created functions:

In [ ]:
def Rx_(theta):
  return np.array([[ 1, 0           , 0           ],
                   [ 0, vf.cos(theta),-vf.sin(theta)],
                   [ 0, vf.sin(theta), vf.cos(theta)]])
  
def Ry_(theta):
  return np.array([[vf.cos(theta), 0, vf.sin(theta)],
                   [ 0           , 1, 0           ],
                   [-vf.sin(theta), 0, vf.cos(theta)]])
  
def Rz_(theta):
  return np.array([[vf.cos(theta), -vf.sin(theta), 0 ],
                   [ vf.sin(theta), vf.cos(theta) , 0 ],
                   [ 0           , 0            , 1 ]])

We create a convieince function to convert from numpy to a list of Variables, and then, we define the lambda function for the rotations:

In [ ]:
np2var = lambda vec: [Variable(x) for x in vec]
x_gt_ad = np2var(x_gt) 
ypr2rot_ad = lambda x: Rz_(x[0]) @ Ry_(x[1]) @ Rx_(x[2])

We can check the function whether it's correctly working. Note that we can nicely use the numpy infrastructure due to the overloads.

In [ ]:
R_ad = ypr2rot_ad(x_gt_ad)
dR_chk = (R_ad - R_gt).astype('float') # Variable ->  conversion works nicely

print('Check rotation composition ( =0): ', np.linalg.norm(dR_chk))

Next we need to define the conversion from 2d numpy float arrays to arrays of Variable 

In [ ]:
def array2var(arr):
    var_arr = np.zeros(arr.shape, dtype=Variable)
    for i in range(arr.shape[0]):
        for j in range(arr.shape[1]):
            var_arr[i, j] = Variable(arr[i, j])
    return var_arr
            
pts_model_ad = array2var(pts_model)  
pts_target_ad = array2var(pts_target)   

Now we have everything to write down the residual vector as a lambda function.

In [ ]:
r_x_ad = lambda x0: ((ypr2rot_ad(x0) @ pts_model.T).T + np.array([x0[3:6]]) - pts_target).ravel()

print('Check residual vector ( =0): ', np.linalg.norm((r_x_ad(x_gt_ad) - r_x(x_gt_ad)).astype('float')))

Note that the residual function will give us back a vector of Variable types, on which we can compute the gradients.

In [ ]:
r_ad = r_x_ad(x_gt_ad)
grad_chk = get_gradients(r_ad[0])

This gradient will be the entries of the Jacobian. Let's compare one entry to the entry of the Jacobian matrix computed earlier using symbolic calculations. 

In [ ]:
J_gt = build_J(x_gt)
print('Check Jacobian entry: ', J_gt[0, 1] - grad_chk[x_gt_ad[1]])

A bit of work, but we can now compose the Jacobian using the auto-diff nano framework. We go through each residual vector element, and then, through each parameter entry.

In [ ]:
def build_J_ad(r, x):
    m = r.shape[0]
    n = 6
    J = np.zeros((m, n))
    for iter_r in range(m):
        grad_iter = get_gradients(r[iter_r])
        for iter_x in range(n):
            J[iter_r, iter_x] = grad_iter[x[iter_x]]
    return J

J_ad = build_J_ad(r_ad, x_gt_ad)
print('Check Jacobian ( =0): ', np.linalg.norm(J_ad - J_gt))

Finally, we can run our Newton-Gauss method.

In [ ]:
def gauss_newton_autodiff_step(x):
    dr = r_x(x)

    # autodiff code
    x_ad = np2var(x)
    dr_ad = r_x_ad(x_ad)
    J_ad = build_J_ad(dr_ad, x_ad)
    
    dx = np.linalg.inv(J_ad.T @ J_ad) @ J_ad.T @ dr
    
    return dx

x0 = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0]) # converging to the solution in 5 iterations
optimizer_framework(gauss_newton_autodiff_step, x0, x_gt)

The iteration steps and results are exactly identical to the case when we used the symbolic library to compute the Jacobian. This code is extremely fast beating most of the earlier implementaions! This solution seems to be the fastest implementation in the notebook so far.

# Changing parametrization: Solution with rotation matrix

In [ ]:
def r_x_12(x, pts_model, pts_target):
    x_R = x[0:9]
    R = x_R.reshape(3,3)
    return ((R @ pts_model.T).T + np.array([x[9:12]]) - pts_target).ravel()

x_gt_12 = np.hstack((R_gt.ravel(), x_gt[3:6]))
print('Check r_x_12 ( =0): ', np.linalg.norm(r_x_12(x_gt_12, pts_model, pts_target)))

In [ ]:
#x0 = np.array([45.0-1.0, -15.0+2.0, 20.0+1.0, 1.0+0.5, -2.0+0.8, 3.0-0.2])
x0 = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
x0[:3] *= 1.0/180.0*math.pi
R0_12 = Rotation.from_euler('ZYX', x0[:3], degrees=False).as_matrix()
x0_12 = np.hstack((R0_12.ravel(), x0[3:6]))

In [ ]:
res = minimize(lambda x: np.linalg.norm(r_x_12(x, pts_model, pts_target)), x0_12, method='trust-constr',
               options={'disp': True, 'maxiter': 2000})

It's less suprising that the optimizer converged, bus it is more supresing that the optimizer actually found the solution! Check it out:

In [ ]:
R_hat_12 = res.x[0:9].reshape(3,3)
print('Final cost: ', np.linalg.norm(r_x_12(res.x, pts_model, pts_target)))
print('Solutions as angles: ', Rotation.from_matrix(R_hat_12).as_euler('ZYX', degrees=True))
print(' ')
print('Compare solution to GT: ( =0)', np.linalg.norm(res.x - x_gt_12))
print('Compare rotation matrix to GT: ( =0)', np.linalg.norm(R_gt - R_hat_12))
print('Check rotation m. orthogonality: ( =0): ', np.linalg.norm(R_hat_12.T @ R_hat_12 - np.eye(3)))
print('Check rotation m. determinant ( =1): ', np.linalg.det(R_hat_12))
print('GT determinant ( =1): ', np.linalg.det(R_gt))

The Euler angles look pretty close to the ground truth, and the orthogonality condition is automatically satisfied without constraints. So I thought this is because the system is overdetermined. Let's see the results when I pick the minimum problem (with the minimum number of points):

In [ ]:
res = minimize(lambda x: np.linalg.norm(r_x_12(x, pts_model[:4], pts_target[:4])), x0_12, method='trust-constr',
               options={'disp': True, 'maxiter': 2000})

R_hat_12 = res.x[0:9].reshape(3,3)

print('Final cost: ', np.linalg.norm(r_x_12(res.x, pts_model[:4], pts_target[:4])))
print('Solutions as angles: ', Rotation.from_matrix(R_hat_12).as_euler('ZYX', degrees=True))
print(' ')
print('Check rotation m. orthogonality: ( =0): ', np.linalg.norm(R_hat_12.T @ R_hat_12 - np.eye(3)))
print('Check rotation m. determinant ( =1): ', np.linalg.det(R_hat_12))

It's quite good! I need to go to an underdetermined case to get non-orthogonal solution for the rotation matrix:

In [ ]:
x0 = np.array([45.0-10.0, -15.0+2.0, 20.0+1.0, 1.0+0.5, -2.0+0.8, 3.0-0.2])
x0[:3] *= 1.0/180.0*math.pi
R0_12 = Rotation.from_euler('ZYX', x0[:3], degrees=False).as_matrix()
x0_12 = np.hstack((R0_12.ravel(), x0[3:6]))

res = minimize(lambda x: np.linalg.norm(r_x_12(x, pts_model[:3], pts_target[:3])), x0_12, method='trust-constr',
               options={'disp': True, 'maxiter': 2000})

R_hat_12 = res.x[0:9].reshape(3,3)

print('Final cost: ', np.linalg.norm(r_x_12(res.x, pts_model[:4], pts_target[:4])))
print('Solutions as angles: ', Rotation.from_matrix(R_hat_12).as_euler('ZYX', degrees=True))
print(' ')
print('Check rotation m. orthogonality: ( =0): ', np.linalg.norm(R_hat_12.T @ R_hat_12 - np.eye(3)))
print('Check rotation m. determinant ( =1): ', np.linalg.det(R_hat_12))

So now I can show-case how to fix this issue with constraints.

In [ ]:
const = ({'type': 'eq', 'fun': lambda x:  (x[0:9].reshape(3,3).T @ x[0:9].reshape(3,3) - np.eye(3))[0,0]},
        {'type': 'eq', 'fun': lambda x:  (x[0:9].reshape(3,3).T @ x[0:9].reshape(3,3) - np.eye(3))[0,1]},
        {'type': 'eq', 'fun': lambda x:  (x[0:9].reshape(3,3).T @ x[0:9].reshape(3,3) - np.eye(3))[0,2]},
        {'type': 'eq', 'fun': lambda x:  (x[0:9].reshape(3,3).T @ x[0:9].reshape(3,3) - np.eye(3))[1,0]},
        {'type': 'eq', 'fun': lambda x:  (x[0:9].reshape(3,3).T @ x[0:9].reshape(3,3) - np.eye(3))[1,1]},
        {'type': 'eq', 'fun': lambda x:  (x[0:9].reshape(3,3).T @ x[0:9].reshape(3,3) - np.eye(3))[1,2]},
        {'type': 'eq', 'fun': lambda x:  (x[0:9].reshape(3,3).T @ x[0:9].reshape(3,3) - np.eye(3))[2,0]},
        {'type': 'eq', 'fun': lambda x:  (x[0:9].reshape(3,3).T @ x[0:9].reshape(3,3) - np.eye(3))[2,1]},
        {'type': 'eq', 'fun': lambda x:  (x[0:9].reshape(3,3).T @ x[0:9].reshape(3,3) - np.eye(3))[2,2]})

res = minimize(lambda x: np.linalg.norm(r_x_12(x, pts_model[:4], pts_target[:4])), x0_12, method='trust-constr',
               options={'disp': True, 'maxiter': 2000}, constraints=const)

R_hat_12 = res.x[0:9].reshape(3,3)

print('Final cost: ', np.linalg.norm(r_x_12(res.x, pts_model[:4], pts_target[:4])))
print('Solutions as angles: ', Rotation.from_matrix(R_hat_12).as_euler('ZYX', degrees=True))
print(' ')
print('Check rotation m. orthogonality: ( =0): ', np.linalg.norm(R_hat_12.T @ R_hat_12 - np.eye(3)))
print('Check rotation m. determinant ( =1): ', np.linalg.det(R_hat_12))

Of course having 9 parameters to describe the rotation is not ideal. For one, Euler parameterization converges faster:

In [ ]:
x0 = np.array([0, 0, 0, 0, 0, 0]) # converges to the solution in 149 iterations
res = minimize(lambda x: np.linalg.norm(r_x(x)), x0, method='trust-constr',
               options={'disp': True, 'maxiter': 2000})

# Solution with Lie algebra

In [ ]:
from scipy.spatial.transform import Rotation

- https://www.ethaneade.com/lie.pdf
- Graph-base SLAM tutorial: http://ais.informatik.uni-freiburg.de/teaching/ws11/robotics2/pdfs/ls-slam-tutorial.pdf

The theme will be the followings:

TODO: this need to be in a reverse direction!

1. We have a known rotation representation (e.g. rotation matrix, quaternion) to simply perform an "action" called rotation on points represented with 3D vectors. However these representations are over-parametrizations of the rotation (for rotation matrix it is 9 and for quaternion it is 4 vs. the minimal representation composed of only 3 parameters). We saw that we needed constraints when using non-minimal rotation representation. To avoid using constraints, We consider special properties of these transformations. These transformations are special mathematical structures called  Lie groups, i.e. SO(3) for rotation matrices and S(4) for quaternions.

2. We will have conversion from the Lie group to another mathematical structure called the Lie algebra. The Lie algebra of rotation matrices is so(3) skew matrices and for quaternions it is S(3) unit quaternions. This conversion or mapping between the group and its algebra is called the logarithmic  map. 

3. The Lie algebra is isomorphic with an n-dimensional vector space. In our case, it means that for each element of so(3) or S(3) I can assign a vector from a 3D vector space. Therefore, we will have a R^3 minimal represention of rotations in the continous vector space. Then, we can optimization without having worried about constraints, however we don't have a direct mathematical formula to apply rotation on a point represented by a vector as we had in the Lie groups.

These steps can be reversed. In this case we need to invert the logarithmic map; this mapping is called exponential map. During optimization we can do both operations to change from one representation to the other depending on whether we perform the rotation or update the optimizer step. 

Dictionary:
- Lie groups: rotation representation with wich we can directly perform rotation, e.g. rotation matrix, quaternion.
- Lie algebra: some intermidiate representation between the Lie group and R^3, e.g. skew matrix for rotation matrix and unit quaternions for quaternions.
- exponential map: conversion
- logarithmic map: conversion
- isomporphic: basically the same space just operations defined a bit differently

In [ ]:
rvec_gt = Rotation.from_euler('ZYX', ypr_gt, degrees=False).as_rotvec(degrees=False)
print('Rotation vector:', rvec_gt)
print('Rotation angle around the axis (omega):', np.linalg.norm(rvec_gt) / pi * 180)
print('Rotation axis: ', rvec_gt / np.linalg.norm(rvec_gt))

The converstion takes two steps: first from vector space to so(3), and then from so(3) to SO(3). I'm following here the lecture notes on [Lie Groups for 2D and 3D Transformations by Ethan Eade](https://www.ethaneade.com/lie.pdf).

Generator system to compute the skew symmetric matrix:

In [ ]:
G1 = np.array([[0, 0, 0], [0, 0, -1], [0, 1, 0]])
G2 = np.array([[0, 0, 1], [0, 0, 0], [-1, 0, 0]])
G3 = np.array([[0, -1, 0], [1, 0, 0], [0, 0, 0]])

Then, we can write the function that converts from rotation vector representation (R^3) to skew symmetric matrix (so(3)):

In [ ]:
skew = lambda w: w[0]*G1 + w[1]*G2 + w[2]*G3

Note that the skew symmetric matrix allows for descring cross product with purely using matrix multiplication. In this sense the skew matrix can be viewed as "changing" the group operation from cross product to matrix-vector multiplication (R^3, x) = (R^3, []_x*) . Since both operations results in the exactly the same results applied on the all R^3 vectors, therefore the two groups are isomorphic.  

In [ ]:
w_a = [-2, 3, 1]
w_b = np.array([4, -8, 2]).reshape(-1)

print('[a]_x * b = ', skew(w_a) @ w_b)
print('a x b =, ', np.cross(w_a, w_b))

Then we can take the skew matrix (so(3)) and apply the exponential map to it; the result is the rotation matrix. The derivation can be found in a lecture notes titled [Lie Groups for 2D and 3D Transformations by Ethan Eade, Page 3-4](https://www.ethaneade.com/lie.pdf). This conversion formula is also known as the Rodrigues' rotation formula, see it on [Wikipedia](https://en.wikipedia.org/wiki/Rodrigues%27_rotation_formula) : 

In [ ]:
omega_gt = np.sqrt(rvec_gt.T @ rvec_gt) # = np.linalg.norm(rvec)
S_gt = skew(rvec_gt)
R_gt_lie = np.eye(3) + (math.sin(omega_gt)/omega_gt)*S_gt + ((1-math.cos(omega_gt))/omega_gt**2)*S_gt@S_gt

In [ ]:
#R_gt = Rotation.from_euler('ZYX', ypr_gt, degrees=False).as_matrix()
print('Check calculation: ', np.linalg.norm(R_gt_lie - R_gt))

Let's visualize these mathematical structures and make sense of it:

In [ ]:
tangent_pt = np.array([1, 0, 0])

# Let's create the surfaces by visualizing their transformation on a 3D vector tangent_pt
SO3_surface = []
so3_surface = []
for yaw in np.arange(-pi, pi, 0.3):                    # sample points at possible Euler 
  for pitch in np.arange(-pi, pi, 0.3):                # angles to construct SO(3)
    for roll in np.arange(-pi, pi, 0.3):
        R = R_x([yaw, pitch, roll])                    # construct rotation matrix SO(3)
        pt_ = R @ tangent_pt                         
        SO3_surface.append(pt_)
        
        omega = np.arccos((np.trace(R)-1)/2)        
        S = (omega/(2*np.sin(omega))) * (R - R.T)      # SO(3) -> so(3)
        pt_ = S @ tangent_pt
        so3_surface.append(pt_)

SO3_surface = np.array(SO3_surface)
so3_surface = np.array(so3_surface)
so3_surface += tangent_pt                              # this will move the points to the identity SO(3) * tangent_pt

# plotting the SO(3) and so(3) surfaces
plt.figure(figsize=(6,5))
ax = plt.axes(projection='3d')
ax.scatter3D(SO3_surface[:,0], SO3_surface[:,1], SO3_surface[:,2], c='r', s=0.1, alpha=0.2)
ax.scatter3D(so3_surface[:,0], so3_surface[:,1], so3_surface[:,2], c='b', s=0.1, alpha=0.2)

# We will create a trajectory in R^3 and show how this trajectory looks like in so(3) and SO(3)
SO3_line = []
so3_line = []
for w_i in np.arange(0.0001, pi, 0.1):
    w = [0, 0, w_i]
    S = np.array([[0, -w[2], w[1]], [w[2], 0, -w[0]], [-w[1], w[0], 0]])            # R^3 -> so(3)
    pt_ = S @ tangent_pt
    so3_line.append(pt_)

    omega = (w[0]**2 + w[1]**2 + w[2]**2)**(1/2)
    R = np.eye(3) + (math.sin(omega)/omega)*S + ((1-math.cos(omega))/omega**2)*S@S  # so(3) -> SO(3)
    pt_ = R @ tangent_pt
    SO3_line.append(pt_)

SO3_line = np.array(SO3_line)
so3_line = np.array(so3_line)
so3_line += tangent_pt 

# plotting the trajectory
ax.plot3D(SO3_line[:,0], SO3_line[:,1], SO3_line[:,2], 'k.-', markersize=6)
ax.plot3D(so3_line[:,0], so3_line[:,1], so3_line[:,2], 'k.-', markersize=6)
set_3d_axes_equal(ax)
ax.view_init(elev=25, azim=120)
ax.set_title('SO(3) Lie group of rotation matrices and\n so(3) Lie algebra of skew matrices')

Let's do exponential map in a more compact form:

In [ ]:
def Exp_(w):
    omega = (w[0]**2 + w[1]**2 + w[2]**2)**(1/2)
    S = np.array([[0, -w[2], w[1]], [w[2], 0, -w[0]], [-w[1], w[0], 0]])
    return np.eye(3) + (math.sin(omega)/omega)*S + ((1-math.cos(omega))/omega**2)*S@S

print('Check exponential map: ', np.linalg.norm(Exp_(rvec_gt) - R_gt))

Reverse direction is easier:

In [ ]:
def Log_(R):
    omega = np.arccos((np.trace(R)-1)/2)           # input rotation matrix
    S = (omega/(2*np.sin(omega))) * (R_gt-R_gt.T)  # rotation matrix SO(3) -> skew so(3)
    return [S[2, 1], S[0, 2], S[1, 0]]             # so(3) -> R^3, using the def. of skew matrix

print('Check log map: ', np.linalg.norm(Exp_(rvec_gt) - R_gt))

In [ ]:
r_x_lie = lambda x0: ((Exp_(x0) @ pts_model.T).T + np.array([x0[3:6]]) - pts_target).ravel()
x_gt_lie = np.hstack((rvec_gt, x_gt[3:6]))
print('Check residual function ( =0): ', np.linalg.norm(r_x_lie(x_gt_lie)))

Let's compare the rotation vector parametrization with the Euler angle implementation using the built-in Nelder-Mead algorithm. As a reminder, here we want to directly solve the cost function, and we need to provide relatively good initial guess for convergence. 

In [ ]:
x0_lie = np.array([45.0-15.0, -15.0+25.0, 20.0+25.0, 1.0+1.5, -2.0+1.8, 3.0-1.2]) # converging to the solution in ~1000 iterations
#x0_lie = np.array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1]) # converging to a local minimum
x0_lie[:3] *= 1.0/180.0*math.pi

print('Solution with Euler angles: ')
res = minimize(lambda x: np.linalg.norm(r_x(x)), x0_lie, method='nelder-mead',
               options={'xatol': 1e-6, 'disp': True, 'maxiter': 2000})
print('Check Euler angle solution ( =0): ', np.linalg.norm(res.x - x_gt))

print(' ')

print('Solution with rotation vectors: ')
x0_lie[:3] = Rotation.from_euler('ZYX', x0_lie[:3], degrees=False).as_rotvec(degrees=False)
res = minimize(lambda x: np.linalg.norm(r_x_lie(x)), x0_lie, method='nelder-mead',
               options={'xatol': 1e-6, 'disp': True, 'maxiter': 2000})

print('Check rotation vector solution ( =0): ', np.linalg.norm(res.x - x_gt_lie))

Both algorithm converged with some initial guess, but starting at very poor initial guess (close to zero), both approaches fail. Seemingly, the rotation vector representation converges faster, which is why this approach is preferred in practice. 

Advantages:
- In general, using rotation vector representation results in faster convergence than the Euler angle representation, and therefore, it's preferred in practice. 
- Only one singular point, when in case of zero rotation, and it can be easily handled.

Drawbacks:
- This approach does not converge with deravtive-free optimizer when the initial guess is very poor. 
- When trying to debug the code, representation of rotation vectors is not easy to interpret. In practice the covariances of the Euler angles are easier to interpret as opposed to the rotation vectors'.

# Operation counts 

In [ ]:
import functools

In [ ]:
class OperationCounterClass(object):
    table = {}

    @staticmethod
    def reset():
        for key in OperationCounter.table:
            OperationCounter.table[key] = 0
    
    @staticmethod
    def incr(op):
        if op in OperationCounter.table.keys():
            OperationCounter.table[op] += 1
        else:
            OperationCounter.table[op] = 1         

    def __repr__(self):
        return str(self.table)

OperationCounter = OperationCounterClass() # so __repr__ works as expected

def operation_counter_decorator(func):
    OperationCounter.table[func.__name__] = 0
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        if len(args) == 1:
            OperationCounter.incr(func.__name__)   
        elif len(args) == 2:
            self = args[0]
            other = args[1]
            if isinstance(other, Variable): # would be called twice if 'other' needs casting
                OperationCounter.incr(func.__name__)   
        else:
            raise NotImplementedError
                        
        return func(*args, **kwargs)
    return wrapper

In [ ]:
class VariableCnt(Variable):
       
    def __init__(self, value, local_gradients=[]):
        if isinstance(value, Variable):  
            Variable.__init__(self, value.value, value.local_gradients)
        else:
            Variable.__init__(self, value, local_gradients)
    
    @operation_counter_decorator
    def __add__(self, other):
        return VariableCnt(Variable.__add__(self, other))
    
    @operation_counter_decorator
    def __mul__(self, other):
        return VariableCnt(Variable.__mul__(self, other))

    @operation_counter_decorator
    def __neg__(self):
        return VariableCnt(Variable.__neg__(self))
    
    #@operation_counter # VariableCnt.var_operation_count['sub'] += 1 # subtraction is composed of neg and add so here we would duplicate count
    def __sub__(self, other):
        return VariableCnt(Variable.__sub__(self, other)) 
    
    __radd__  = __add__
    __rmul__ = __mul__
    __rsub__ = __sub__
    
class VarFunctionsCnt(VarFunctions):
    @staticmethod
    @operation_counter_decorator
    def sin(a):
        return VariableCnt(VarFunctions.sin(a))
    
    @staticmethod
    @operation_counter_decorator
    def cos(a):
        return VariableCnt(VarFunctions.cos(a))

vf = VarFunctionsCnt

In [ ]:
OperationCounter.reset()

test_a = VariableCnt(1)
test_b = VariableCnt(2)
test_c = (test_a - vf.cos(test_b))*2

print('Operation types and counts: ', OperationCounter)
print('Value at the last variable ( !=0): ', test_c.value)
print('Gradients at the last variable (= not []): ', test_c.local_gradients)

In [ ]:
def Rx_(theta):
  return np.array([[ 1, 0           , 0           ],
                   [ 0, vf.cos(theta),-vf.sin(theta)],
                   [ 0, vf.sin(theta), vf.cos(theta)]])
  
def Ry_(theta):
  return np.array([[vf.cos(theta), 0, vf.sin(theta)],
                   [ 0           , 1, 0           ],
                   [-vf.sin(theta), 0, vf.cos(theta)]])
  
def Rz_(theta):
  return np.array([[vf.cos(theta), -vf.sin(theta), 0 ],
                   [ vf.sin(theta), vf.cos(theta) , 0 ],
                   [ 0           , 0            , 1 ]])

np2var = lambda vec, typeOf=VariableCnt: [typeOf(x) for x in vec]
ypr2rot_ad = lambda x: Rz_(x[0]) @ Ry_(x[1]) @ Rx_(x[2])
r_x_ad = lambda x0: ((ypr2rot_ad(x0) @ pts_model.T).T + np.array([x0[3:6]]) - pts_target).ravel()

Run a Gauss-Newton just to check that everything's ok...

In [ ]:
x0 = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0]) # converging to the solution in 5 iterations

def gauss_newton_autodiff_count_step(x):
    x_ad = np2var(x)
    dr_ad = r_x_ad(x_ad)
    dr_f = dr_ad.astype('float')
    J_ad = build_J_ad(dr_ad, x_ad)
    
    dx = np.linalg.inv(J_ad.T @ J_ad) @ J_ad.T @ dr_f    
    return dx

optimizer_framework(gauss_newton_autodiff_count_step, x0, x_gt)

In [ ]:
OperationCounter.reset()

x0_ad = np2var(x0)
dr_ad = r_x_ad(x0_ad)
J_ad = build_J_ad(dr_ad, x0_ad)

print('To be sure that type is correct ( =True): ', type(dr_ad[0]) is VariableCnt)
print('Operations counts:', OperationCounter) # {'add': 154, 'mul': 136, 'neg': 3, 'sin': 6, 'cos': 6}

# Automatic differentation with rotation vectors

In [ ]:
class VariableExt(VariableCnt):
            
    def __init__(self, value, local_gradients=[]):
        VariableCnt.__init__(self, value, local_gradients)
    
    def __pow__(self, other):
        if isinstance(other, Variable) and other.value == 2: 
            OperationCounter.incr('pow2')
        elif isinstance(other, Variable) and other.value == 0.5: 
            OperationCounter.incr('sqrt')
        elif isinstance(other, Variable) and other.value < 1.0: 
            OperationCounter.incr('invpow')
        elif isinstance(other, Variable):
            OperationCounter.incr('pow')
        
        if not isinstance(other, Variable):
            return self.__pow__(VariableExt(float(other)))                
        value = self.value ** other.value
        local_gradients = ((self,  other.value*self.value**(other.value-1) ), 
                           (other, self.value**other.value*math.log(other.value)))
        return VariableExt(value, local_gradients)
             
    @operation_counter_decorator        
    def __inv__(self):
        value = 1. / self.value
        local_gradients = (
            (self, -1 / self.value**2),
        )
        return VariableExt(value, local_gradients)
        
    def __truediv__(self, other):
        if not isinstance(other, Variable):
            return self.__truediv__(VariableExt(float(other)))         
        return self.__mul__(other.__inv__()) 

    def __add__(self, other):
        return VariableExt(VariableCnt.__add__(self, other))
    
    def __mul__(self, other):
        return VariableExt(VariableCnt.__mul__(self, other))

    def __neg__(self):
        return VariableExt(VariableCnt.__neg__(self))
    
    def __sub__(self, other):
        return VariableExt(VariableCnt.__sub__(self, other)) 
    
    __radd__  = __add__
    __rmul__ = __mul__
    __rsub__ = __sub__
        
class VarFunctionsExt(VarFunctionsCnt):

    @staticmethod
    def sin(a):
        if not isinstance(a, Variable):
            return VarFunctionsExt.sin(VariableExt(float(a)))                
        return VariableExt(VarFunctionsCnt.sin(a))
    
    @staticmethod
    def cos(a):
        if not isinstance(a, Variable):
            return VarFunctionsExt.cos(VariableExt(float(a)))                
        return VariableExt(VarFunctionsCnt.cos(a))

vf = VarFunctionsExt
_var = VariableExt

In [ ]:
OperationCounter.reset()

test_a = _var(3)
test_b = _var(4)
test_c = (test_a**2 + test_b**2)**(_var(1)/_var(2))
print('Test value ( =5): ', test_c.value)
print('Local gradient ( not =[]): ', test_c.local_gradients)

print('Number of operations: ', OperationCounter)

In [ ]:
x_gt_al = np.hstack((rvec_gt, x_gt[3:6]))

def Exp_al_(w):
    omega = (w[0]**2 + w[1]**2 + w[2]**2)**(1/2) # some modification here, because we need to use elementary operations
    S = np.array([[0, -w[2], w[1]], [w[2], 0, -w[0]], [-w[1], w[0], 0]])
    # need to change the scalar * matrix order to matrix * scalar & convert all constants to VariableExt
    R = np.eye(3) + S*(vf.sin(omega)/omega) + S@S*((_var(1)-vf.cos(omega))/omega**2)
    return R

R_gt_al = Exp_al_(np2var(x_gt_al, typeOf=VariableExt))
print(np.linalg.norm((R_gt_al - R_gt).astype('float')))
print(R_gt_al[0,0].local_gradients)
print(type(R_gt_al[0, 0]))

In [ ]:
r_x_al = lambda x0: ((Exp_al_(x0) @ pts_model.T).T + np.array([x0[3:6]]) - pts_target).ravel()
print('Check residual function ( =0): ', np.linalg.norm(r_x_al(x_gt_al).astype('float')))

In [ ]:
def gauss_newton_rvec_step(x):
    x_al = np2var(x, typeOf=VariableExt)
    dr_al = r_x_al(x_al)
    dr_f = dr_al.astype('float')
    J_al = build_J_ad(dr_al, x_al)
    dx = np.linalg.inv(J_al.T @ J_al) @ J_al.T @ dr_f
    return dx

x0 = np.array([0.001, 0, 0, 0.0, 0.0, 0.0]) # converging to the solution in 5 iterations
optimizer_framework(gauss_newton_rvec_step, x0, x_gt_al)

In [ ]:
OperationCounter.reset()

x0_al = np2var(x0, typeOf=VariableExt)
dr_al = r_x_al(x0_al)
J_al = build_J_ad(dr_al, x0_al)

print('To be sure that type is correct ( =True): ', type(dr_ad[0]) is VariableExt)
print('Operation count: ', OperationCounter)
# Euler angles were: {'add': 154, 'mul': 136, 'neg': 3, 'sin': 6, 'cos': 6}

TODO: rotation matrix unknowns with lie algebra and automatic differentation

# Timeit

In [ ]:
%%timeit -r 5 -n 10
cog_model = np.mean(pts_model, axis=0)
cog_target = np.mean(pts_target, axis=0)
pts_model_cog = pts_model - cog_model
pts_target_cog = pts_target - cog_target
C = pts_model_cog.T @ pts_target_cog
[U, S, V] = np.linalg.svd(C) # this will be always the svd of a 3x3 matrix independently the number of points
R_svd = V.T @ U.T
t_svd = cog_target - R_svd @ cog_model

In [ ]:
%%timeit -r 5 -n 10
x0 = np.array([0.001, 0, 0, 0.0, 0.0, 0.0])
dx = np.inf 
iter_num = 0
while np.linalg.norm(dx) > 1e-6 and iter_num < 100:
    iter_num += 1
    x0_al = np2var(x0, typeOf=VariableExt)
    dr_al = r_x_al(x0_al)
    J_al = build_J_ad(dr_al, x0_al)
    dr_f = dr_al.astype('float')
    #dx = np.linalg.inv(J_al.T @ J_al) @ J_al.T @ dr_f
    dx = np.linalg.solve(J_al.T @ J_al, J_al.T @ dr_f)
    x0 = x0 - dx

In [ ]:
%%timeit -r 5 -n 10
x0 = np.array([0, 0, 0, 0, 0, 0])
res = least_squares(r_x, x0, verbose=0, xtol=1e-6)